In [1]:
import pandas as pd

In [2]:
final_df = pd.read_json('data/final_data.json')

In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Community Area Number                         52 non-null     int64  
 1   COMMUNITY AREA NAME                           52 non-null     object 
 2   PERCENT OF HOUSING CROWDED                    52 non-null     float64
 3   PERCENT HOUSEHOLDS BELOW POVERTY              52 non-null     float64
 4   PERCENT AGED 16+ UNEMPLOYED                   52 non-null     float64
 5   PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA  52 non-null     float64
 6   PERCENT AGED UNDER 18 OR OVER 64              52 non-null     float64
 7   PER CAPITA INCOME                             52 non-null     int64  
 8   HARDSHIP INDEX                                52 non-null     int64  
 9   community                                     52 non-null     objec

In [3]:
final_df.describe()

,Community Area Number,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,NATIVE,Area,PER CAPITA INCOME IN K,CountPerSqkm,shannon_index
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,32.788462,4.634615,20.992308,13.534615,18.600000,33.753846,28923.230769,43.211538,5286.576923,14.395468,28.923231,426.006618,5.281253
std,21.316786,3.532587,11.500569,6.916700,11.511563,7.939735,16998.418673,28.794973,12132.445038,7.829675,16.998419,1093.630232,2.277910
min,1.000000,0.300000,3.400000,4.700000,2.500000,13.500000,8201.000000,1.000000,3.000000,2.831664,8.201000,0.152124,1.039721
25%,14.750000,1.975000,12.750000,8.700000,11.450000,27.175000,16856.250000,18.500000,88.500000,9.226535,16.856250,7.269774,3.578140
50%,30.500000,3.700000,18.700000,11.000000,17.350000,35.650000,23865.000000,40.500000,699.000000,13.667303,23.865000,80.497067,5.433673
75%,51.250000,6.300000,29.050000,17.525000,24.475000,39.125000,35818.000000,65.250000,4988.000000,16.817997,35.818000,312.025967,7.071402
max,77.000000,15.200000,56.500000,34.600000,54.800000,51.500000,88669.000000,98.000000,75132.000000,50.651008,88.669000,6863.622752,9.626278


In [5]:
final_df.head(15)

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,community,SCIENTIFIC NAME,NATIVE,COUNT,Area,PER CAPITA INCOME IN K,CountPerSqkm,PovertyFlag,shannon_index
0,1,ROGERS PARK,7.7,23.6,8.7,18.2,27.5,23939,39,ROGERS PARK,"[Spinus tristis, Turdus migratorius, Turdus mi...",676,"[1, 46, 4, 9, 1, 3, 1, 9, 12, 1, 2, 1, 2, 1, 7...",8.631616,23.939,78.316735,Poor,5.498251
1,2,WEST RIDGE,7.8,17.2,8.8,20.8,38.5,23040,46,WEST RIDGE,"[Sturnus vulgaris, Aquila chrysaetos, Passer d...",255,"[7, 1, 5, 4, 100, 2, 2, 2, 1, 1, 1, 25, 25, 25...",16.570330,23.040,15.388951,Poor,4.148305
2,3,UPTOWN,3.8,24.0,8.9,11.8,22.2,35787,20,UPTOWN,"[Corvus brachyrhynchos, Corvus brachyrhynchos,...",75132,"[5, 7, 3, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, ...",10.946406,35.787,6863.622752,Poor,9.626278
3,4,LINCOLN SQUARE,3.4,10.9,8.2,13.4,25.5,37524,17,LINCOLN SQUARE,"[Spinus tristis, Acanthis flammea, Junco hyema...",3751,"[1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",12.002108,37.524,312.528443,Poor,7.116321
4,5,NORTH CENTER,0.3,7.5,5.2,4.5,26.2,57123,6,NORTH CENTER,"[Corvus brachyrhynchos, Spinus tristis, Setoph...",1432,"[1, 7, 1, 28, 22, 4, 2, 5, 2, 1, 3, 2, 6, 2, 1...",9.588837,57.123,149.340322,Rich,6.540097
5,6,LAKE VIEW,1.1,11.4,4.7,2.6,17.0,60058,5,LAKE VIEW,"[Corvus brachyrhynchos, Corvus brachyrhynchos,...",13334,"[1, 2, 1, 1, 1, 1, 1, 1, 7, 100, 8, 1, 1, 42, ...",14.656079,60.058,909.793133,Rich,7.465087
6,7,LINCOLN PARK,0.8,12.3,5.1,3.6,21.5,71551,2,LINCOLN PARK,"[Corvus brachyrhynchos, Corvus brachyrhynchos,...",31120,"[6, 20, 11, 2, 2, 3, 1, 8, 4, 291, 5, 1, 39, 1...",14.831229,71.551,2098.275219,Rich,9.101900
7,8,NEAR NORTH SIDE,1.9,12.9,7.0,2.5,22.6,88669,1,NEAR NORTH SIDE,"[Corvus brachyrhynchos, Toxostoma rufum, Brant...",1654,"[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, ...",12.867114,88.669,128.544749,Rich,5.314266
8,10,NORWOOD PARK,2.0,5.4,9.0,11.5,39.5,32875,21,NORWOOD PARK,"[Loxia leucoptera, Spinus tristis, Sturnus vul...",153,"[15, 3, 2, 1, 20, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1...",20.521535,32.875,7.455583,Poor,3.915351
9,11,JEFFERSON PARK,2.7,8.6,12.4,13.4,35.5,27751,25,JEFFERSON PARK,"[Spinus tristis, Poecile atricapillus, Dryobat...",52,"[2, 4, 1, 9, 1, 5, 2, 8, 2, 1, 2, 2, 2, 2, 25,...",10.912843,27.751,4.765028,Poor,3.230655


In [24]:
# Selecting features and target variable
features = ['PERCENT OF HOUSING CROWDED', 
            'PERCENT HOUSEHOLDS BELOW POVERTY', 
#            'PERCENT AGED 16+ UNEMPLOYED',
#            'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 
#            'PERCENT AGED UNDER 18 OR OVER 64',
            'HARDSHIP INDEX',
            'PER CAPITA INCOME']

target = 'shannon_index'

# Splitting the data into training and testing sets
X = final_df[features]
y = final_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


# Initializing and training the Decision Tree Regressor model
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = tree_model.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (Decision Trees):", mse)

Mean Squared Error (Decision Trees): 13.741316951922567


In [27]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initializing and training the KNN regressor model
knn_model = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors as needed
knn_model.fit(X_train_scaled, y_train)

# Predicting on the test set
y_pred_knn = knn_model.predict(X_test_scaled)

# Evaluating the model
mse_knn = mean_squared_error(y_test, y_pred_knn)
print("Mean Squared Error (KNN):", mse_knn)

Mean Squared Error (KNN): 4.415633525599799


In [28]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (Linear Regression):", mse_linear)

Mean Squared Error (Linear Regression): 4.346668043530402


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report

# Initializing and training the Random Forest classifier
rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Making predictions on the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluating the model
mse_random = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (Random Forest):", mse_random)

Mean Squared Error (Random Forest): 4.290840329025155


In [ ]:
# features = ['PERCENT OF HOUSING CROWDED', 
#             'PERCENT HOUSEHOLDS BELOW POVERTY', 
#             'PERCENT AGED 16+ UNEMPLOYED', 
#             'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 
#             'PERCENT AGED UNDER 18 OR OVER 64',
#             'HARDSHIP INDEX',
#             'PER CAPITA INCOME']
#Mean Squared Error (Decision Trees): 14.384432122771436
#Mean Squared Error (KNN): 4.139463044176787
#Mean Squared Error (Linear Regression): 5.303329023877021
#Mean Squared Error (Random Forest): 4.368482085574034

In [ ]:
# features = ['PERCENT OF HOUSING CROWDED', 
#             'PERCENT HOUSEHOLDS BELOW POVERTY', 
#             'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 
#             'PERCENT AGED UNDER 18 OR OVER 64',
#             'HARDSHIP INDEX',
#             'PER CAPITA INCOME']
#Mean Squared Error (Decision Trees): 12.213353087465187
#Mean Squared Error (KNN): 4.117839844458724
#Mean Squared Error (Linear Regression): 4.763298189159179
#Mean Squared Error (Random Forest): 4.2496426590700525

In [ ]:
# features = ['PERCENT OF HOUSING CROWDED', 
#             'PERCENT HOUSEHOLDS BELOW POVERTY', 
#             'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 
#             'HARDSHIP INDEX',
#             'PER CAPITA INCOME']
#Mean Squared Error (Decision Trees): 7.32619548931322
#Mean Squared Error (KNN): 4.9332521262880284
#Mean Squared Error (Linear Regression): 4.4638652250276145
#Mean Squared Error (Random Forest): 4.830146615875486

In [ ]:
# features = ['PERCENT OF HOUSING CROWDED', 
#             'PERCENT HOUSEHOLDS BELOW POVERTY', 
#             'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA', 
#             'HARDSHIP INDEX',
#             'PER CAPITA INCOME']
#Mean Squared Error (Decision Trees): 13.741316951922567
#Mean Squared Error (KNN): 4.415633525599799
#Mean Squared Error (Linear Regression): 4.346668043530402
#Mean Squared Error (Random Forest): 4.290840329025155